In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import torch as tc
genders = ['female', 'male']

In [2]:
their_semantics = [
    'Subject',
    'Measuring station',
    'repetition no.',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'subject',
    'measures_station',
    'repetition',
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

station_list = ["SS20", "Expert", "Our"]

measures = pd.read_pickle(f'data/ss20_cleaned_measures.zip')
measures = measures[their_semantics]
measures.columns = our_semantic
our_measures = pd.read_pickle("data/ss20_our_measures.zip")
measures = pd.concat([our_measures, measures]).drop(columns=["gender", "stature"])
measures = measures[measures['measures_station'].isin(["SS20", "Expert measurer 1", "our"])]
measures.replace("Expert measurer 1", "Expert", inplace=True)
measures.replace("our", "Our", inplace=True)

# CALC ICC

In [3]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure)['ICC'].loc[2]
        measures_of_station.append(icc)
    all_measures.append(measures_of_station)
our_icc = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [4]:
a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.906992,20.503528,58,59,4.489077e-24,"[0.85, 0.94]"
1,ICC2,Single random raters,0.907037,20.718765,58,58,7.171410e-24,"[0.85, 0.94]"
2,ICC3,Single fixed raters,0.907914,20.718765,58,58,7.171410e-24,"[0.85, 0.94]"
3,ICC1k,Average raters absolute,0.951228,20.503528,58,59,4.489077e-24,"[0.92, 0.97]"
4,ICC2k,Average random raters,0.951253,20.718765,58,58,7.171410e-24,"[0.92, 0.97]"
5,ICC3k,Average fixed raters,0.951735,20.718765,58,58,7.171410e-24,"[0.92, 0.97]"


In [5]:
our_icc.index = ['Expert', "SS20", 'Our']
our_icc

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist
Expert,0.992658,0.983699,0.981579,0.976319,0.986107,0.980247,0.870923
SS20,0.996636,0.992145,0.766478,0.783158,0.969863,0.987054,0.907914
Our,0.997617,0.994610,0.993753,0.981880,0.990776,0.992513,0.965700


# CALC SEM

In [6]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')['ICC'].loc[2]
        sem = a[measure].std() * np.sqrt(1-icc)
        measures_of_station.append(sem)
    all_measures.append(measures_of_station)
our_sem = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [7]:
our_sem.index = ['Expert', "SS20", 'Our']
our_sem.T.round(4)

,Expert,SS20,Our
bust_chest_girth,10.9598,7.2411,6.6435
waist_girth,17.3445,13.0130,10.3207
hip_girth,15.1953,49.4566,7.8820
thigh_girth,11.0017,32.7144,9.1193
upper_arm_girth,5.0882,7.4577,4.4127
neck_girth,5.9301,5.7025,4.3156
neck_waist,10.6182,10.6424,7.5959


# CALC PSD

In [8]:
def calculate_psd(measure):
    data = measures
    f1 = data['repetition'] == 1
    f2 = data['repetition'] == 2
    data = data[ f1 | f2 ][['subject', 'measures_station', 'repetition', measure]]
    data = data.pivot_table(values=measure, index=['subject', 'repetition'], columns="measures_station")
    data = data.dropna()
    

    column_wise = []
    for column1 in data.columns:
        row_wise = []
        for column2 in data.columns:
            if column1 == column2:
                row_wise.append(np.nan)
            else:
                d1 = data[column1]
                d2 = data[column2]
                var = (d1-d2).var()
                sem1 = our_sem[measure].loc[column1]**2
                sem2 = our_sem[measure].loc[column2]**2
                value = np.sqrt(var + (sem1/2) + (sem2/2))
                row_wise.append(value)
        column_wise.append(row_wise)
    data = pd.DataFrame(column_wise, index=data.columns, columns=data.columns)
    data.index.name = None
    return data

In [9]:
neck_result = calculate_psd(measure='neck_girth')
MX = pd.Series({"SS20/Expert": neck_result.loc["SS20"]["Expert"]})
neck_result = neck_result.loc['Our'].drop('Our').append(MX)

In [10]:
bust_result = calculate_psd(measure='bust_chest_girth')
MX = pd.Series({"SS20/Expert": bust_result.loc["SS20"]["Expert"]})
bust_result = bust_result.loc['Our'].drop('Our').append(MX)

In [11]:
waist_result = calculate_psd(measure='waist_girth')
MX = pd.Series({"SS20/Expert": waist_result.loc["SS20"]["Expert"]})
waist_result = waist_result.loc['Our'].drop('Our').append(MX)

In [12]:
hip_result = calculate_psd(measure='hip_girth')
MX = pd.Series({"SS20/Expert": hip_result.loc["SS20"]["Expert"]})
hip_result = hip_result.loc['Our'].drop('Our').append(MX)

In [13]:
arm_result = calculate_psd(measure='upper_arm_girth')
MX = pd.Series({"SS20/Expert": arm_result.loc["SS20"]["Expert"]})
arm_result = arm_result.loc['Our'].drop('Our').append(MX)

In [14]:
thigh_result = calculate_psd(measure='thigh_girth')
MX = pd.Series({"SS20/Expert": thigh_result.loc["SS20"]["Expert"]})
thigh_result = thigh_result.loc['Our'].drop('Our').append(MX)

In [15]:
neck_waist_result = calculate_psd(measure='neck_waist')
MX = pd.Series({"SS20/Expert": neck_waist_result.loc["SS20"]["Expert"]})
neck_waist_result = neck_waist_result.loc['Our'].drop('Our').append(MX)

In [16]:
def calculate_mad():
    all_measures = []
    for station in station_list:
        measures_of_station = []
        a = measures[measures['measures_station'] == station]
        f1 = a['repetition'] == 1
        f2 = a['repetition'] == 2
        for measure in our_semantic[3:-1]:
            mean = np.nanmean(abs(a[f1][measure].to_numpy() - a[f2][measure].to_numpy()))
            measures_of_station.append(mean)
        all_measures.append(measures_of_station)
    return pd.DataFrame(all_measures, columns=our_semantic[3:-1], index=['SS20','Expert','Our']).T

In [17]:
styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "125%"),
                              ("color", 'black'),
                              ("font-weight", '700')])]

In [18]:
mad_table = calculate_mad().round(4)
mad_table = mad_table.style.set_caption("OUR vs SS20 MAD statistic result").set_table_styles(styles)
mad_table.format(precision=4)

,SS20,Expert,Our
bust_chest_girth,7.6308,10.1818,8.0104
waist_girth,7.9020,16.2273,11.4546
hip_girth,7.1314,13.5000,19.0181
thigh_girth,6.0895,12.5909,13.4123
upper_arm_girth,4.5074,5.0000,8.3640
neck_girth,4.5742,6.5455,5.9050
neck_waist,7.4790,11.6364,9.3180


In [19]:
sem_table = our_sem.T.round(4)
sem_table = sem_table.style.set_caption("OUR vs SS20 SEM statistic result").set_table_styles(styles)
sem_table.format(precision=4)

,Expert,SS20,Our
bust_chest_girth,10.9598,7.2411,6.6435
waist_girth,17.3445,13.0130,10.3207
hip_girth,15.1953,49.4566,7.8820
thigh_girth,11.0017,32.7144,9.1193
upper_arm_girth,5.0882,7.4577,4.4127
neck_girth,5.9301,5.7025,4.3156
neck_waist,10.6182,10.6424,7.5959


In [20]:
data_psd = pd.DataFrame(
    [neck_result, bust_result, waist_result, hip_result, arm_result,thigh_result, neck_waist_result],
    index=our_semantic[3:-1]).round(2)
data_psd.columns = ['Our vs Expert', "Our vs SS20", 'SS20 vs Expert']
data_psd.to_csv("output/our_psd.csv")
data_psd = data_psd.style.set_caption("OUR vs SS20 PSD statistic result").set_table_styles(styles)
data_psd.format(precision=2)

,Our vs Expert,Our vs SS20,SS20 vs Expert
bust_chest_girth,24.30,16.59,23.68
waist_girth,33.11,22.42,36.04
hip_girth,63.99,40.61,54.98
thigh_girth,63.90,67.11,47.07
upper_arm_girth,13.62,31.96,33.05
neck_girth,41.11,42.92,31.62
neck_waist,33.78,33.25,32.72
